In [18]:
from google.colab import drive
drive.mount('/content/drive')

PATH = '/content/drive/MyDrive/MITB/4 2023 Aug/ISSS609 Text Analytics and Applications/AnalyzingDisneylandReviews'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Sentiment Analysis
https://www.kaggle.com/code/ganiesenov/sentiment-analysis-using-nltk-gensim-models

ABSA
* https://github.com/yangheng95/PyABSA/blob/v2/examples-v2/aspect_polarity_classification/Aspect_Sentiment_Classification.ipynb
* https://www.kaggle.com/code/phiitm/aspect-based-sentiment-analysis
* https://www.kaggle.com/code/nkitgupta/aspect-based-sentiment-analysis

In [22]:
# Required libraries in Colab
! pip install transformers sentencepiece contractions num2words --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [23]:
import sys

import numpy as np
import pandas as pd
import re
from tqdm import tqdm, tqdm_pandas

import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)
import plotly.express as px
from wordcloud import WordCloud

import nltk
nltk.download('stopwords')
from nltk.corpus import PlaintextCorpusReader, stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer


from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

## 1. Load data

### 1.1 Import data

In [24]:
# Import data
df = pd.read_pickle(PATH+'/data/processed_reviews.pkl')

df.head()

,Review_ID,Sentence_ID,Review_Year,Review_Month,Branch,Rating,Reviewer_Location,Review_Title,Review_Text
0,1,1,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,I visited Disney Land Tokyo with my family on a weekend night in December 2022.
1,1,2,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,We bought the evening entry that allowed us to enter the park after 3 p.m. at a discounted rate.
2,1,3,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,"We thought it was a great deal because we could still enjoy most of the attractions, parades, and shows without spending too much time or money."
3,1,4,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,We arrived at the park around 4 p.m. and headed straight to Tokyo Disneyland.
4,1,5,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,We were amazed by the beautiful decorations and the festive atmosphere.


### 1.2 Vectorize
This incorporates the text preprocessing steps in `text_preprocessing.py`

In [30]:
sys.path.append(PATH+'/notebooks')
from text_preprocessing import text_preprocessing

column_names = ["Review_Text"]  # each column will take around 30s to process

# For storing the TF-IDF vectors
tfidf_vectors_dict = {}

# For storing the features (words)
feature_names_dict = {}

for column_name in column_names:

    # Vectorize
    vectorizer = TfidfVectorizer(analyzer = lambda x: text_preprocessing(x,
                                                                        stemmer = SnowballStemmer(language='english'),
                                                                        # lemmatizer = WordNetLemmatizer(),
                                                                        stopword_list = stopwords.words('english')
                                                                        ),
                                max_df = 0.5,     # cannot appear in more than half of the docs
                                min_df = 2        # must appear in at least 2 docs
                                )

    X = vectorizer.fit_transform(df[column_name])
    tfidf_vectors_dict[f'{column_name}_tfidf'] = X

    feature_names = vectorizer.get_feature_names_out()
    feature_names_dict[f'{column_name}_features'] = feature_names

    print(f"### {column_name} ###")
    print("n_samples: %d, n_features: %d" % X.shape)
    print(f"features: {feature_names}")
    print()


    # Bag of Words
    df[f"{column_name}_BOW"] =  df[column_name].apply(lambda x : text_preprocessing(x,
                                                                                    stemmer = SnowballStemmer(language='english'),
                                                                                    # lemmatizer = WordNetLemmatizer(),
                                                                                    stopword_list = stopwords.words('english')))



df.head(5)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Review_Text ###
n_samples: 88918, n_features: 6343
features: ['aa' 'abandon' 'abat' ... 'zone' 'zoom' 'zootopia']



,Review_ID,Sentence_ID,Review_Year,Review_Month,Branch,Rating,Reviewer_Location,Review_Title,Review_Text,Review_Text_BOW
0,1,1,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,I visited Disney Land Tokyo with my family on a weekend night in December 2022.,"[visit, disney, land, tokyo, famili, weekend, night, decemb]"
1,1,2,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,We bought the evening entry that allowed us to enter the park after 3 p.m. at a discounted rate.,"[bought, even, entri, allow, us, enter, park, three, discount]"
2,1,3,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,"We thought it was a great deal because we could still enjoy most of the attractions, parades, and shows without spending too much time or money.","[thought, great, deal, could, still, enjoy, show, without, spend, much, time]"
3,1,4,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,We arrived at the park around 4 p.m. and headed straight to Tokyo Disneyland.,"[arriv, park, around, four, head, straight, tokyo]"
4,1,5,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,We were amazed by the beautiful decorations and the festive atmosphere.,"[amaz, beauti, decor, festiv]"


In [31]:
# Unpack BOW
df["Review_Text_BOW"] = df["Review_Text_BOW"].apply(lambda x : ' '.join(x))

In [32]:
# TF-IDF vectors for Review_Text
tfidf = tfidf_vectors_dict['Review_Text_tfidf']

tfidf

<88918x6343 sparse matrix of type '<class 'numpy.float64'>'
	with 599376 stored elements in Compressed Sparse Row format>

# 2. Sentiment Analysis

### 2.1 VADER

In [34]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

df["vader_compound_score"] = df["Review_Text_BOW"].apply(lambda x : sia.polarity_scores(x)['compound'])

df.head(10)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


,Review_ID,Sentence_ID,Review_Year,Review_Month,Branch,Rating,Reviewer_Location,Review_Title,Review_Text,Review_Text_BOW,vader_compound_score
0,1,1,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,I visited Disney Land Tokyo with my family on a weekend night in December 2022.,visit disney land tokyo famili weekend night decemb,0.0000
1,1,2,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,We bought the evening entry that allowed us to enter the park after 3 p.m. at a discounted rate.,bought even entri allow us enter park three discount,0.2263
2,1,3,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,"We thought it was a great deal because we could still enjoy most of the attractions, parades, and shows without spending too much time or money.",thought great deal could still enjoy show without spend much time,0.8074
3,1,4,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,We arrived at the park around 4 p.m. and headed straight to Tokyo Disneyland.,arriv park around four head straight tokyo,0.2263
4,1,5,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,We were amazed by the beautiful decorations and the festive atmosphere.,amaz beauti decor festiv,0.0000
5,1,6,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,We had a wonderful time at Disney Land Tokyo at night with our family.,wonder time disney land tokyo night,0.0000
6,1,7,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,We felt that it was worth every penny and every minute.,felt worth everi penni everi,0.2263
7,1,8,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,We would definitely recommend it to anyone who wants to experience the best of both parks in a short time.,would definit recommend anyon want experi best park short,0.7906
8,1,9,2023,9,Disneyland_Tokyo,5,"Johor Bahru, Malaysia",Worth every penny and every minute,It was a memorable visit that we will never forget.,memor visit never,0.0000
9,2,10,2023,9,Disneyland_Tokyo,5,"Perth, Australia",The BEST day in Tokyo Disney,Honestly this was a brilliant day at Tokyo Disney.,honest brilliant day tokyo,0.7964


### 2.2 BERT

In [35]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification


tokenizer = AutoTokenizer.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")
model = AutoModelForSequenceClassification.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")

classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device_map="auto")